<a href="https://colab.research.google.com/github/yamadashamoji/00_tools/blob/main/hateful_words_filtering_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 悪意のある言葉フィルタリングアプリ(簡易版)

# 依存ライブラリのインストール（Colabで最初に実行）
!pip install -q transformers torch ipywidgets fugashi unidic-lite huggingface_hub

import re
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import ipywidgets as widgets
from IPython.display import display, HTML
from getpass import getpass
from huggingface_hub import login

# Hugging Faceトークンの入力（不要な場合もあり）
# print("Hugging Faceのアクセストークンを入力してください（公開モデルの場合はEnterでスキップ）:")
# hf_token = getpass() or None
# if hf_token:
#     login(token=hf_token)  # トークンで認証

# 禁止ワードリスト（フレーズを追加）
prohibited_words = ["バカ", "アホ", "殴る", "殺す", "バカすぎる", "ムカつく"]

# BERTモデルとトークナイザーのロード
model_name = "kit-nlp/bert-base-japanese-sentiment-cyberbullying"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, token=hf_token)
except Exception as e:
    print(f"モデル '{model_name}' のロードに失敗しました: {e}")
    print("トークンが正しいか、リポジトリへのアクセス権があるか確認してください。")
    raise e

# 感情分析関数
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).squeeze().tolist()
    # kit-nlp/bert-base-japanese-sentiment-cyberbullying: 0=ネガティブ, 1=ポジティブ
    return {"positive": probabilities[1], "negative": probabilities[0]}

# テキストを文に分割
def split_into_sentences(text):
    sentences = re.split(r'[。！？\n]+', text)
    return [s.strip() for s in sentences if s.strip()]

# 禁止ワードを置き換え
def replace_prohibited_words(text, prohibited_words):
    replaced_text = text
    replaced_words = []
    for word in prohibited_words:
        if word in replaced_text:
            replaced_text = replaced_text.replace(word, "[見せられないよ]")
            replaced_words.append(word)
    return replaced_text, replaced_words

# メイン処理
def process_text(input_text):
    sentences = split_into_sentences(input_text)
    processed_sentences = []
    all_replaced_words = []
    detailed_log = []

    for sentence in sentences:
        # 感情分析
        sentiment = analyze_sentiment(sentence)
        negative_prob = sentiment["negative"]
        # フィルタリング条件: 禁止ワードを含む場合、閾値を0.5に下げる
        threshold = 0.5 if any(word in sentence for word in prohibited_words) else 0.7
        if negative_prob >= threshold and any(word in sentence for word in prohibited_words):
            processed_sentence, replaced_words = replace_prohibited_words(sentence, prohibited_words)
            all_replaced_words.extend(replaced_words)
            # 詳細ログに文脈情報を追加
            detailed_log.append(
                f"文: {sentence}<br>"
                f"ネガティブ確率: {negative_prob:.2f}（閾値: {threshold}）<br>"
                f"置き換えた単語: {', '.join(replaced_words) if replaced_words else 'なし'}<br>"
                f"処理後: {processed_sentence}<br>"
            )
        else:
            processed_sentence = sentence
            # ネガティブ確率が低い場合の理由を追加
            reason = "フィルタリング不要（ネガティブ確率が閾値未満または禁止ワードなし）"
            if negative_prob < threshold:
                reason += f"<br>注: このモデルはネットいじめ特有の強い攻撃的表現に敏感です。軽度の侮辱（例: 'バカ'）はネガティブとみなされない場合があります。"
            detailed_log.append(
                f"文: {sentence}<br>"
                f"ネガティブ確率: {negative_prob:.2f}（閾値: {threshold}）<br>"
                f"処理: {reason}<br>"
            )
        processed_sentences.append(processed_sentence)

    # 結果を結合
    result_text = "。".join(processed_sentences)
    if result_text and not result_text.endswith("。"):
        result_text += "。"

    # ログ出力（HTMLで色付け）
    result_text_html = result_text.replace("[見せられないよ]", "<span style='color:red'>[見せられないよ]</span>")
    log = f"<b>処理結果:</b> {result_text_html}<br>"
    if all_replaced_words:
        log += f"<b>置き換えた単語（全体）:</b> {', '.join(set(all_replaced_words))}<br>"
    else:
        log += "<b>置き換えた単語（全体）:</b> なし<br>"
    log += "<b>詳細ログ:</b><br>" + "<hr>".join(detailed_log)

    return log

# UIウィジェット
input_box = widgets.Textarea(
    value="",
    placeholder="ここにテキストを入力してください（例: お前バカすぎるよ。バカでも分かる説明。）",
    description="入力:",
    layout={'width': '500px', 'height': '100px'}
)
output_area = widgets.HTML(value="")
process_button = widgets.Button(description="処理開始")

def on_button_clicked(b):
    input_text = input_box.value
    if input_text:
        result = process_text(input_text)
        output_area.value = result
    else:
        output_area.value = "<b>エラー:</b> テキストを入力してください"

process_button.on_click(on_button_clicked)

# UI表示
display(HTML("<h3>悪意のある言葉フィルタリングアプリ</h3>"))
display(input_box)
display(process_button)
display(output_area)

悪意のある言葉フィルタリングアプリ 仕様書（正規版）
1. 概要
本アプリは、ユーザーが入力した日本語テキストをリアルタイムで分析し、悪意のある言葉や不適切な表現を検出して「見せられないよ」などのメッセージに置き換えるWebアプリケーションです。感情分析にはBERT（cl-tohoku/bert-base-japanese）を用い、特定の禁止ワードリストに基づくフィルタリングを組み合わせます。
2. 目的

ユーザーが入力するテキストから悪意のある言葉を検出し、適切な代替表現に置き換える。
日本語の感情分析を通じて、ネガティブな感情を含む文脈を高精度に識別。
安全で快適なコミュニケーション環境を提供。

3. 機能要件
3.1 テキスト入力と処理

入力: ユーザーがテキストボックスに日本語テキストを入力。
出力: 処理後のテキストをリアルタイムで表示。悪意のある言葉は「見せられないよ」に置き換え。
処理速度: 入力から1秒以内に結果を表示。

3.2 感情分析

モデル: transformersライブラリとcl-tohoku/bert-base-japaneseを使用。
分類: テキストをポジティブ/ネガティブ/ニュートラルに分類。
閾値: ネガティブと判定された確率が0.7以上の場合、悪意のある可能性が高いとみなす。
対象: 文単位で分析（長編テキストは文に分割して処理）。

3.3 禁止ワードフィルタリング

ワードリスト: 事前に定義された禁止ワードリスト（例: 侮辱、差別、暴力的な表現）を使用。
処理: 感情分析でネガティブと判定された文に含まれる禁止ワードを「見せられないよ」に置き換え。
カスタマイズ: ユーザーがワードリストを追加/編集可能（管理画面経由）。

3.4 ユーザーインターフェース

テキストボックス: ユーザーがテキストを入力するエリア。
結果表示エリア: 処理後のテキストを表示。
フィードバック: 置き換えが発生した場合、どの部分が置き換えられたかをハイライト表示。
管理画面: 禁止ワードリストの編集やモデルの設定変更。

4. 非機能要件

パフォーマンス: 最大512トークンのテキストを1秒以内に処理。
スケーラビリティ: クラウド環境（例: AWS, GCP）でのデプロイを想定し、同時ユーザー100人に対応。
セキュリティ: 入力テキストは一時保存のみで、処理後に削除。禁止ワードリストは暗号化して保存。
互換性: 主要ブラウザ（Chrome, Firefox, Safari）で動作。

5. 技術スタック

フロントエンド: React, Tailwind CSS（軽量でモダンなUI）。
バックエンド: Python (FastAPI)、transformers、PyTorch。
モデル: cl-tohoku/bert-base-japanese（日本語BERT）。
デプロイ: Dockerコンテナ、クラウド（AWS/GCP）。
データベース: SQLite（禁止ワードリスト管理用、軽量）。

6. 処理フロー

ユーザーがテキストを入力。
テキストを文単位に分割（句点や改行で分割）。
各文をBERTモデルで感情分析（ポジティブ/ネガティブ/ニュートラル）。
ネガティブと判定された文に対し、禁止ワードリストと照合。
該当するワードを「見せられないよ」に置き換え。
処理済みテキストをUIに表示（置き換え部分はハイライト）。

7. リスクと課題

感情分析の精度: BERTモデルの精度に依存。誤検知/未検知を減らすため、追加のファインチューニングが必要。
禁止ワードの網羅性: リストの更新頻度や方言・スラングへの対応が課題。
処理速度: 長編テキストや高負荷時のパフォーマンス確保。
文化的ニュアンス: 日本語特有の表現や文脈の解釈ミスを防ぐ。

8. 拡張性

多言語対応（他のBERTモデルを追加）。
リアルタイムチャットアプリへの組み込み。
学習済みモデルの定期的な更新と再学習。

9. 開発スケジュール（仮）

Week 1-2: 環境構築、BERTモデルとtransformersのセットアップ。
Week 3-4: 感情分析と禁止ワードフィルタリングの実装。
Week 5: UI開発（React + Tailwind CSS）。
Week 6: 統合テスト、デプロイ準備。
Week 7: ユーザーテスト、フィードバック反映。

10. 付録
10.1 禁止ワードリストの例

侮辱: 「バカ」「アホ」
差別: （具体例は倫理的配慮から省略）
暴力: 「殴る」「殺す」

10.2 サンプル入力と出力
入力: 「お前バカすぎるよ、ほんとムカつく。」出力: 「お前[見せられないよ]すぎるよ、ほんと[見せられないよ]。」
